In [1]:
import numpy as np 
import warnings
warnings.filterwarnings("ignore", category = np.VisibleDeprecationWarning) 
allDB = {}

In [2]:
readDB = np.genfromtxt("myfile.csv", dtype = None, delimiter = '|', names=True, autostrip=True)
allDB["R"] = readDB
allDB["R"].dtype

dtype([('name', 'S5'), ('age', '<i8')])

In [5]:
column = "age"
col = allDB["R"][column] 
mean = np.mean(col)
print(mean)
print("The Mean for the column " + column + " is " + str(mean))

32.25
The Mean for the column age is 32.25
